day2dayのデータに整形（形式を変更）

In [22]:
import pandas as pd
import numpy as np 
import os

整形済みデータを編集する

In [43]:
df_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_finalinput.csv', header=None)
df_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_finalinput_hour.csv', header=None)
print(df_act)

         0    1    2  3    4    5    6    7    8    9   ...   74   75   76  \
0     23002  <b>  3.0  5    2    3    2    1  <e>  <b>  ...  <p>  <p>  <p>   
1     23002  <b>  2.0  1    3    2    1  <e>  <b>    3  ...  <p>  <p>  <p>   
2     23003  <b>  3.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>   
3     23003  <b>  2.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>   
4     23003  <b>  2.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>   
...     ...  ...  ... ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3987  21402  <b>  3.0  2    1  <e>  <b>  955  <e>  <b>  ...  <p>  <p>  <p>   
3988  21402  <b>  2.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>   
3989  21415  <b>  2.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>   
3990  21457  <b>  3.0  5    1  <e>  <b>  955  <e>  <b>  ...  <p>  <p>  <p>   
3991  21550  <b>  2.0  5    2    1  <e>  <b>  955  <e>  ...  <p>  <p>  <p>   

       77   78   79   80   81   82   83  
0     <p>  <p>  <p>  

In [45]:
# 各行見ていって，<b>を消す＆<e>を<S>に置換
df_act = df_act.replace('<e>', '<s>')
df_time = df_time.replace('<e>', '<s>')

# 特定の文字を削除して左詰め（文字列結合で再構成）
def remove_and_shift(row, char='<b>'):
    # 各セルから特定文字を削除し、行を再構成
    cleaned = [str(cell).replace(char, '') for cell in row]
    # 空でないものを左に寄せ、空文字を右に詰める
    non_empty = [c for c in cleaned if c]
    empty = [''] * (len(row) - len(non_empty))
    return pd.Series(non_empty + empty)

# 各行に適用
df_act_cleaned = df_act.apply(remove_and_shift, axis=1)
df_time_cleaned = df_time.apply(remove_and_shift, axis=1)

df_act_cleaned = df_act_cleaned.fillna('<p>')
df_time_cleaned = df_time_cleaned.fillna('<p>')

# print(df_act_cleaned)

df_act_cleaned = df_act_cleaned.replace(['', 'NaN'], '<p>') #, inplace=True)
df_time_cleaned = df_time_cleaned.replace(['', 'NaN'], '<p>') #, inplace=True)

print(df_act_cleaned)


         0    1  2    3    4    5    6    7    8    9   ...   74   75   76  \
0     23002  3.0  5    2    3    2    1  <s>    3    2  ...  <p>  <p>  <p>   
1     23002  2.0  1    3    2    1  <s>    3    2    3  ...  <p>  <p>  <p>   
2     23003  3.0  1  <s>  955  <s>  955  <s>    3    1  ...  <p>  <p>  <p>   
3     23003  2.0  1  <s>  955  <s>  955  <s>  955  <s>  ...  <p>  <p>  <p>   
4     23003  2.0  1  <s>  955  <s>  955  <s>  955  <s>  ...  <p>  <p>  <p>   
...     ...  ... ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3987  21402  3.0  2    1  <s>  955  <s>  955  <s>  955  ...  <p>  <p>  <p>   
3988  21402  2.0  1  <s>  955  <s>  955  <s>  955  <s>  ...  <p>  <p>  <p>   
3989  21415  2.0  1  <s>  955  <s>  955  <s>  955  <s>  ...  <p>  <p>  <p>   
3990  21457  3.0  5    1  <s>  955  <s>  955  <s>  955  ...  <p>  <p>  <p>   
3991  21550  2.0  5    2    1  <s>  955  <s>  955  <s>  ...  <p>  <p>  <p>   

       77   78   79   80   81   82   83  
0     <p>  <p>  <p>  

In [46]:
print(df_act_cleaned.iloc[2, :].unique())

['23003' '3.0' '1' '<s>' '955' '3' '2' '<p>']


In [47]:
df_act_cleaned.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_finalinput_cleaned.csv')
df_time_cleaned.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_finalinput_cleaned.csv')

In [ ]:
print(df_act.head(4))

# 対象の文字
target_char = '<SEP>'

# 行単位の処理
def remove_from_last_char(row, char):
    # 行全体を1つの文字列として連結
    combined = ''.join(str(x) for x in row)
    # 最後の対象文字の位置を見つける
    last_idx = combined.rfind(char)
    if last_idx == -1:
        cleaned = combined  # 対象文字がなければそのまま
    else:
        cleaned = combined[:last_idx]  # それ以降を削除

    # 左詰めに分割（元の列数で埋め直す）
    cells = list(cleaned)[:len(row)]
    padded = cells + [''] * (len(row) - len(cells))
    return pd.Series(padded)

# 各行に適用
df_cleaned = df.apply(lambda row: remove_from_last_char(row, target_char), axis=1)

print(df_cleaned)

      0    1    2  3    4    5    6    7    8    9   ...   74   75   76   77  \
0  23002  <b>  3.0  5    2    3    2    1  <e>  <b>  ...  <p>  <p>  <p>  <p>   
1  23002  <b>  2.0  1    3    2    1  <e>  <b>    3  ...  <p>  <p>  <p>  <p>   
2  23003  <b>  3.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>  <p>   
3  23003  <b>  2.0  1  <e>  <b>  955  <e>  <b>  955  ...  <p>  <p>  <p>  <p>   

    78   79   80   81   82   83  
0  <p>  <p>  <p>  <p>  <p>  <p>  
1  <p>  <p>  <p>  <p>  <p>  <p>  
2  <p>  <p>  <p>  <p>  <p>  <p>  
3  <p>  <p>  <p>  <p>  <p>  <p>  

[4 rows x 84 columns]


In [20]:
# print(df_act.head(5))
# print(df_time.head(5))
print(type(df_act.iloc[:, 2]))
print(df_act.iloc[:, 2].dtype)
print(df_act.iloc[:, 2])
df_act.iloc[:, 2] = df_act.iloc[:, 2].astype(int)

print(df_act.iloc[:, 2])
print(df_act.iloc[:, 2].unique().dtype)
print(df_act.iloc[:, 3].unique().dtype)
print(df_act.iloc[:, 1].unique().dtype)

print(df_act.iloc[:, 2].dtype)


<class 'pandas.core.series.Series'>
float64
0       3.0
1       2.0
2       3.0
3       2.0
4       2.0
       ... 
3987    3.0
3988    2.0
3989    2.0
3990    3.0
3991    2.0
Name: 2, Length: 3992, dtype: float64
0       3.0
1       2.0
2       3.0
3       2.0
4       2.0
       ... 
3987    3.0
3988    2.0
3989    2.0
3990    3.0
3991    2.0
Name: 2, Length: 3992, dtype: float64
float64
object
object
float64


In [31]:
xlsx_basepath = "/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/q_rawans_202212" # WEBアンケート_0124_0130.xlsx
# Q1: 1219, Q2: 0108, Q3: 0124 
xl_q3 = pd.ExcelFile(os.path.join(xlsx_basepath, "WEBアンケート_0124_0130.xlsx"))
xl_q2 = pd.ExcelFile(os.path.join(xlsx_basepath, "WEBアンケート_0107_0108.xlsx"))
xl_q1 = pd.ExcelFile(os.path.join(xlsx_basepath, "WEBアンケート_1219_1225.xlsx"))
df_q3 = xl_q3.parse()
df_q2 = xl_q2.parse()
df_q1 = xl_q1.parse()
df_q3.to_csv(os.path.join(xlsx_basepath, "WEBアンケート_0124_0130.csv"), index=False)
df_q2.to_csv(os.path.join(xlsx_basepath, "WEBアンケート_0107_0108.csv"), index=False)
df_q1.to_csv(os.path.join(xlsx_basepath, "WEBアンケート_1219_1225.csv"), index=False)

In [32]:
df3_columns = df_q3.columns
df2_columns = df_q2.columns
df1_columns = df_q1.columns
# print(df_columns)
# print(df2_columns)

df3_filtered = df_q3[['調査期間中に1回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に2回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に3回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に4回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に5回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に6回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に7回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に8回目に渋谷駅を訪れた日付を教えてください．']] #, '調査期間中に9回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に10回目に渋谷駅を訪れた日付を教えてください．']]
df2_filtered = df_q2[['調査期間中に1回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に2回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に3回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に4回目に渋谷駅を訪れた日付を教えてください．']] # '調査期間中に5回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に6回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に7回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に8回目に渋谷駅を訪れた日付を教えてください．']] #, '調査期間中に9回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に10回目に渋谷駅を訪れた日付を教えてください．']]
df1_filtered = df_q1[['調査期間中に1回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に2回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に3回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に4回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に5回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に6回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に7回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に8回目に渋谷駅を訪れた日付を教えてください．']] #, '調査期間中に9回目に渋谷駅を訪れた日付を教えてください．', '調査期間中に10回目に渋谷駅を訪れた日付を教えてください．']]
# print(df_filtered)
# df_filteredでNANではない行の数を取得
# print(df_filtered.columns)
print(df3_filtered.shape[0])
for column in df3_filtered.columns:
    # 各列のNaNでない行の数を取得
    non_nan_count = df3_filtered[column].notna().sum()
    print(f"{column}: {non_nan_count}")

print(df2_filtered.shape[0])
for column in df2_filtered.columns:
    # 各列のNaNでない行の数を取得
    non_nan_count = df2_filtered[column].notna().sum()
    print(f"{column}: {non_nan_count}")

print(df1_filtered.shape[0])
for column in df1_filtered.columns:
    # 各列のNaNでない行の数を取得
    non_nan_count = df1_filtered[column].notna().sum()
    print(f"{column}: {non_nan_count}")

145
調査期間中に1回目に渋谷駅を訪れた日付を教えてください．: 145
調査期間中に2回目に渋谷駅を訪れた日付を教えてください．: 145
調査期間中に3回目に渋谷駅を訪れた日付を教えてください．: 123
調査期間中に4回目に渋谷駅を訪れた日付を教えてください．: 62
調査期間中に5回目に渋谷駅を訪れた日付を教えてください．: 54
調査期間中に6回目に渋谷駅を訪れた日付を教えてください．: 38
調査期間中に7回目に渋谷駅を訪れた日付を教えてください．: 24
調査期間中に8回目に渋谷駅を訪れた日付を教えてください．: 14
161
調査期間中に1回目に渋谷駅を訪れた日付を教えてください．: 161
調査期間中に2回目に渋谷駅を訪れた日付を教えてください．: 161
調査期間中に3回目に渋谷駅を訪れた日付を教えてください．: 137
調査期間中に4回目に渋谷駅を訪れた日付を教えてください．: 15
153
調査期間中に1回目に渋谷駅を訪れた日付を教えてください．: 153
調査期間中に2回目に渋谷駅を訪れた日付を教えてください．: 153
調査期間中に3回目に渋谷駅を訪れた日付を教えてください．: 139
調査期間中に4回目に渋谷駅を訪れた日付を教えてください．: 67
調査期間中に5回目に渋谷駅を訪れた日付を教えてください．: 56
調査期間中に6回目に渋谷駅を訪れた日付を教えてください．: 41
調査期間中に7回目に渋谷駅を訪れた日付を教えてください．: 24
調査期間中に8回目に渋谷駅を訪れた日付を教えてください．: 16


tripデータのみを取り出すか？（東京PTの要領で，day to day情報を付加）

In [45]:
import sys
base_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/04_202212old'
folderlist = os.listdir(base_path)
user_log_dict = {}
user_count_dict = {} # 何日登場したかのカウント
user_time_traj = {} # key: userid, value: time trajectory
user_act_traj = {} # key: userid, value: act trajectory
nokitaku_count = 0
all_count = 0
folderlist = sorted(folderlist)
print(folderlist)

for folder in folderlist:
    if not os.path.isdir(os.path.join(base_path, folder)):
        continue
    # print(folder)
    filelist = os.listdir(os.path.join(base_path, folder))
    for file in filelist:
        if file.endswith('trip.csv'):
            # print(folder, file)
            df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
            df['出発時刻'] = pd.to_datetime(df['出発時刻']) #, format='%Y/%m/%d %H:%M:%S')
            df['到着時刻'] = pd.to_datetime(df['到着時刻']) #, format='%Y/%m/%d %H:%M:%S')

            grouped = df.groupby('ユーザーID')
            for user_id, group in grouped:
                userid = group['ユーザーID'].values[0]
                group = group.sort_values(by='出発時刻')
                acts = list(group['目的コード（active）'])
                times = list(group['出発時刻'])
                # print('type time', type(times), times)
                # print('type act', type(acts), acts)
                # sys.exit()

                all_count += 1
                if acts[-1] != 110:
                    if acts[-1] == 998 or acts[-1] == 999:
                        acts[-1] = 110
                    # print('kitakushitenai!!!')
                if acts[-1] != 110:
                    nokitaku_count += 1
                    continue
                if acts[0] == 110:
                    nokitaku_count += 1
                    continue
                times = list(group['出発時刻'].dt.floor('h'))

                acts.insert(0, '<b>')
                acts.append('<e>')
                times.insert(0, '<b>')
                times.append('<e>')

                if user_id not in list(user_log_dict.keys()):
                    user_log_dict[user_id] = len(group)
                    user_count_dict[user_id] = 1
                    user_time_traj[user_id] = times
                    user_act_traj[user_id] = acts

                else:
                    user_log_dict[user_id] += len(group)
                    user_count_dict[user_id] += 1
                    user_time_traj[user_id] += times
                    user_act_traj[user_id] += acts

print('nokitaku_count', nokitaku_count)
print('all_count', all_count)

print(user_log_dict, len(user_log_dict))
print(user_count_dict, len(user_count_dict))

new_dict = {key: value // 5 for key, value in user_count_dict.items()}
print(new_dict)
print(sum(new_dict.values()))
print('user_log_dict', user_act_traj)
print('user_time_traj', user_time_traj)
len_values = list(user_time_traj.values())
max_len = max(len(value) for value in len_values)
print('max_len', max_len)
df_time_traj_df = pd.DataFrame.from_dict(user_time_traj, orient='index')
# index（＝ユーザーID）を新しい列 'user_id' として追加

# print('df_time_traj_df', df_time_traj_df)
df_act_traj_df = pd.DataFrame.from_dict(user_act_traj, orient='index')
df_act_traj_df.insert(0, 'user_id', df_act_traj_df.index)
df_time_traj_df.insert(0, 'user_id', df_time_traj_df.index)
# print('df_act_traj_df', df_act_traj_df)

# df_act_traj_df.to_csv(os.path.join(base_path, 'user_act_traj_df.csv'), index=False)
df_act_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog01.csv', index=False)
df_time_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog01.csv', index=False)

# for key, value in user_time_traj.items():
#     print(len(value))

    # print('user_id', key)
    # print('user_time_traj', value)
    # print('user_act_traj', user_act_traj[key])
    # sys.exit()

['.DS_Store', '10028_20221212', '10028_20221213', '10028_20221214', '10028_20221215', '10028_20221216', '10028_20221217', '10028_20221218', '10028_20221219', '10028_20221220', '10028_20221221', '10028_20221222', '10028_20221223', '10028_20221224', '10028_20221225', '10028_20230107', '10028_20230108', '10028_20230109', '10028_20230110', '10028_20230111', '10028_20230112', '10028_20230113', '10028_20230124', '10028_20230125', '10028_20230126', '10028_20230127', '10028_20230128', '10028_20230129', '10028_20230130']
nokitaku_count 628
all_count 3088
{23001: 72, 23002: 58, 23003: 19, 23005: 122, 23010: 42, 23014: 31, 23023: 72, 23024: 62, 23027: 65, 23034: 53, 23036: 24, 23037: 6, 23040: 49, 23042: 66, 23044: 26, 23045: 40, 23047: 20, 23058: 105, 23059: 61, 23060: 14, 23062: 45, 23063: 36, 23064: 86, 23069: 27, 23070: 30, 23071: 36, 23073: 49, 23076: 14, 23080: 34, 23084: 95, 23086: 24, 23091: 62, 23092: 34, 23093: 68, 23095: 22, 23096: 43, 23099: 44, 23103: 39, 23105: 51, 23107: 136, 23108

23新規メンバー

In [46]:
base_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/04_202301new'
folderlist = os.listdir(base_path)
folderlist = sorted(folderlist)
user_log_dict = {}
user_count_dict = {} # 何日登場したかのカウント
user_time_traj = {} # key: userid, value: time trajectory
user_act_traj = {} # key: userid, value: act trajectory
nokitaku_count = 0
all_count = 0
folderlist = sorted(folderlist)
print(folderlist)

for folder in folderlist:
    if not os.path.isdir(os.path.join(base_path, folder)):
        continue
    # print(folder)
    filelist = os.listdir(os.path.join(base_path, folder))
    for file in filelist:
        if file.endswith('trip.csv'):
            # print(folder, file)
            df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
            df['出発時刻'] = pd.to_datetime(df['出発時刻']) #, format='%Y/%m/%d %H:%M:%S')
            df['到着時刻'] = pd.to_datetime(df['到着時刻']) #, format='%Y/%m/%d %H:%M:%S')

            grouped = df.groupby('ユーザーID')
            for user_id, group in grouped:
                userid = group['ユーザーID'].values[0]
                group = group.sort_values(by='出発時刻')
                acts = list(group['目的コード（active）'])
                times = list(group['出発時刻'])
                # print('type time', type(times), times)
                # print('type act', type(acts), acts)
                # sys.exit()

                all_count += 1
                if acts[-1] != 110:
                    if acts[-1] == 998 or acts[-1] == 999:
                        acts[-1] = 110
                    # print('kitakushitenai!!!')
                if acts[-1] != 110:
                    nokitaku_count += 1
                    continue
                if acts[0] == 110:
                    nokitaku_count += 1
                    continue
                times = list(group['出発時刻'].dt.floor('h'))

                acts.insert(0, '<b>')
                acts.append('<e>')
                times.insert(0, '<b>')
                times.append('<e>')

                if user_id not in list(user_log_dict.keys()):
                    user_log_dict[user_id] = len(group)
                    user_count_dict[user_id] = 1
                    user_time_traj[user_id] = times
                    user_act_traj[user_id] = acts

                else:
                    user_log_dict[user_id] += len(group)
                    user_count_dict[user_id] += 1
                    user_time_traj[user_id] += times
                    user_act_traj[user_id] += acts

print('nokitaku_count', nokitaku_count)
print('all_count', all_count)

print(user_log_dict, len(user_log_dict))
print(user_count_dict, len(user_count_dict))

new_dict = {key: value // 5 for key, value in user_count_dict.items()}
print(new_dict)
print(sum(new_dict.values()))
print('user_log_dict', user_act_traj)
print('user_time_traj', user_time_traj)
len_values = list(user_time_traj.values())
max_len = max(len(value) for value in len_values)
print('max_len', max_len)
df_time_traj_df = pd.DataFrame.from_dict(user_time_traj, orient='index')
# print('df_time_traj_df', df_time_traj_df)
df_act_traj_df = pd.DataFrame.from_dict(user_act_traj, orient='index')
df_act_traj_df.insert(0, 'user_id', df_act_traj_df.index)
df_time_traj_df.insert(0, 'user_id', df_time_traj_df.index)
# print('df_act_traj_df', df_act_traj_df)

# df_act_traj_df.to_csv(os.path.join(base_path, 'user_act_traj_df.csv'), index=False)
df_act_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog02.csv', index=False)
df_time_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog02.csv', index=False)

# user_log_dict = {}
# user_count_dict = {} # 何日登場したかのカウント
# for folder in folderlist:
#     if not os.path.isdir(os.path.join(base_path, folder)):
#         continue
#     # print(folder)
#     filelist = os.listdir(os.path.join(base_path, folder))
#     for file in filelist:
#         if file.endswith('trip.csv'):
#             print(folder, file)
#             df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
#             grouped = df.groupby('ユーザーID')
#             for user_id, group in grouped:
#                 if user_id not in list(user_log_dict.keys()):
#                     user_log_dict[user_id] = len(group)
#                     user_count_dict[user_id] = 1
#                 else:
#                     user_log_dict[user_id] += len(group)
#                     user_count_dict[user_id] += 1

# print(user_log_dict, len(user_log_dict))
# print(user_count_dict, len(user_count_dict))

# new_dict = {key: value // 5 for key, value in user_count_dict.items()}
# print(new_dict)
# print(sum(new_dict.values()))
# print('sum::::', sum(list(new_dict.values())))


['.DS_Store', '10029_20230107', '10029_20230108', '10029_20230109', '10029_20230110', '10029_20230111', '10029_20230112', '10029_20230113', '10029_20230124', '10029_20230125', '10029_20230126', '10029_20230127', '10029_20230128', '10029_20230129', '10029_20230130']
nokitaku_count 6
all_count 46
{23251: 9, 23252: 2, 23255: 12, 23256: 9, 23259: 15, 23260: 43, 23253: 23, 23254: 9, 23257: 9} 9
{23251: 4, 23252: 1, 23255: 3, 23256: 3, 23259: 4, 23260: 14, 23253: 4, 23254: 4, 23257: 3} 9
{23251: 0, 23252: 0, 23255: 0, 23256: 0, 23259: 0, 23260: 2, 23253: 0, 23254: 0, 23257: 0}
2
user_log_dict {23251: ['<b>', 200, 110, '<e>', '<b>', 200, 110, '<e>', '<b>', 200, 110, '<e>', '<b>', 210, 210, 110, '<e>'], 23252: ['<b>', 998, 110, '<e>'], 23255: ['<b>', 200, 110, 998, 110, '<e>', '<b>', 210, 210, 200, 110, '<e>', '<b>', 300, 200, 300, 110, '<e>'], 23256: ['<b>', 200, 110, '<e>', '<b>', 310, 300, 110, '<e>', '<b>', 300, 310, 210, 110, '<e>'], 23259: ['<b>', 998, 210, 210, 110, '<e>', '<b>', 300, 9

In [47]:
base_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/05_202311'
folderlist = os.listdir(base_path)
folderlist = sorted(folderlist)
user_log_dict = {}
user_count_dict = {} # 何日登場したかのカウント
user_time_traj = {} # key: userid, value: time trajectory
user_act_traj = {} # key: userid, value: act trajectory
nokitaku_count = 0
all_count = 0
folderlist = sorted(folderlist)
print(folderlist)

for folder in folderlist:
    if not os.path.isdir(os.path.join(base_path, folder)):
        continue
    # print(folder)
    filelist = os.listdir(os.path.join(base_path, folder))
    for file in filelist:
        if file.endswith('trip.csv'):
            # print(folder, file)
            df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
            df['出発時刻'] = pd.to_datetime(df['出発時刻']) #, format='%Y/%m/%d %H:%M:%S')
            df['到着時刻'] = pd.to_datetime(df['到着時刻']) #, format='%Y/%m/%d %H:%M:%S')

            grouped = df.groupby('ユーザーID')
            for user_id, group in grouped:
                userid = group['ユーザーID'].values[0]
                group = group.sort_values(by='出発時刻')
                acts = list(group['目的コード（active）'])
                times = list(group['出発時刻'])
                # print('type time', type(times), times)
                # print('type act', type(acts), acts)
                # sys.exit()

                all_count += 1
                if acts[-1] != 110:
                    if acts[-1] == 998 or acts[-1] == 999:
                        acts[-1] = 110
                    # print('kitakushitenai!!!')
                if acts[-1] != 110:
                    nokitaku_count += 1
                    continue
                if acts[0] == 110:
                    nokitaku_count += 1
                    continue
                times = list(group['出発時刻'].dt.floor('h'))

                acts.insert(0, '<b>')
                acts.append('<e>')
                times.insert(0, '<b>')
                times.append('<e>')

                if user_id not in list(user_log_dict.keys()):
                    user_log_dict[user_id] = len(group)
                    user_count_dict[user_id] = 1
                    user_time_traj[user_id] = times
                    user_act_traj[user_id] = acts

                else:
                    user_log_dict[user_id] += len(group)
                    user_count_dict[user_id] += 1
                    user_time_traj[user_id] += times
                    user_act_traj[user_id] += acts

print('nokitaku_count', nokitaku_count)
print('all_count', all_count)

print(user_log_dict, len(user_log_dict))
print(user_count_dict, len(user_count_dict))

new_dict = {key: value // 5 for key, value in user_count_dict.items()}
print(new_dict)
print(sum(new_dict.values()))
print('user_log_dict', user_act_traj)
print('user_time_traj', user_time_traj)
len_values = list(user_time_traj.values())
max_len = max(len(value) for value in len_values)
print('max_len', max_len)
df_time_traj_df = pd.DataFrame.from_dict(user_time_traj, orient='index')
# print('df_time_traj_df', df_time_traj_df)
df_act_traj_df = pd.DataFrame.from_dict(user_act_traj, orient='index')
df_act_traj_df.insert(0, 'user_id', df_act_traj_df.index)
df_time_traj_df.insert(0, 'user_id', df_time_traj_df.index)
# print('df_act_traj_df', df_act_traj_df)

# df_act_traj_df.to_csv(os.path.join(base_path, 'user_act_traj_df.csv'), index=False)
df_act_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog03.csv', index=False)
df_time_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog03.csv', index=False)

# user_log_dict = {}
# user_count_dict = {} # 何日登場したかのカウント
# for folder in folderlist:
#     if not os.path.isdir(os.path.join(base_path, folder)):
#         continue
#     # print(folder)
#     filelist = os.listdir(os.path.join(base_path, folder))
#     for file in filelist:
#         if file.endswith('trip.csv'):
#             print(folder, file)
#             df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
#             grouped = df.groupby('ユーザーID')
#             for user_id, group in grouped:
#                 if user_id not in list(user_log_dict.keys()):
#                     user_log_dict[user_id] = len(group)
#                     user_count_dict[user_id] = 1
#                 else:
#                     user_log_dict[user_id] += len(group)
#                     user_count_dict[user_id] += 1

# print(user_log_dict, len(user_log_dict))
# print(user_count_dict, len(user_count_dict))

# new_dict = {key: value // 5 for key, value in user_count_dict.items()}
# print(new_dict)
# print('sum::::', sum(list(new_dict.values())))


['.DS_Store', '10032_20231115', '10032_20231116', '10032_20231117', '10032_20231118', '10032_20231119', '10032_20231120', '10032_20231121', '10032_20231122', '10032_20231123', '10032_20231124', '10032_20231125', '10032_20231126', '10032_20231127', '10032_20231128', '10032_20231129', '10032_20231130', '10032_20231201', '10032_20231202']
nokitaku_count 739
all_count 3263
{23603: 42, 23604: 35, 23606: 56, 23607: 52, 23609: 46, 23610: 36, 23614: 66, 23615: 44, 23616: 42, 23617: 39, 23622: 40, 23623: 128, 23625: 27, 23630: 55, 23636: 44, 23643: 35, 23644: 54, 23646: 73, 23649: 30, 23652: 38, 23654: 30, 23656: 8, 23657: 66, 23663: 39, 23664: 40, 23666: 84, 23667: 50, 23669: 45, 23673: 36, 23674: 47, 23675: 65, 23677: 23, 23679: 66, 23682: 58, 23683: 21, 23684: 31, 23686: 29, 23687: 64, 23688: 57, 23689: 55, 23693: 18, 23694: 70, 23695: 64, 23696: 42, 23701: 41, 23702: 27, 23703: 56, 23704: 10, 23706: 14, 23708: 30, 23711: 41, 23716: 58, 23718: 36, 23720: 41, 23721: 49, 23724: 23, 23726: 44, 

江東区2019PPも利用

In [48]:
import sys
import pandas as pd
import numpy as np
import os
base_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/toyosu_2019/201907-202002'
folderlist = os.listdir(base_path)
user_log_dict = {}
user_count_dict = {} # 何日登場したかのカウント
user_time_traj = {} # key: userid, value: time trajectory
user_act_traj = {} # key: userid, value: act trajectory
nokitaku_count = 0
all_count = 0
folderlist = sorted(folderlist)
print(folderlist)

for folder in folderlist:
    if not os.path.isdir(os.path.join(base_path, folder)):
        continue
    # print(folder)
    filelist = os.listdir(os.path.join(base_path, folder))
    for file in filelist:
        if file.endswith('trip.csv'):
            # print(folder, file)
            df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
            df['出発時刻'] = pd.to_datetime(df['出発時刻']) #, format='%Y/%m/%d %H:%M:%S')
            df['到着時刻'] = pd.to_datetime(df['到着時刻']) #, format='%Y/%m/%d %H:%M:%S')

            grouped = df.groupby('ユーザーID')
            for user_id, group in grouped:
                userid = group['ユーザーID'].values[0]
                group = group.sort_values(by='出発時刻')
                acts = list(group['目的コード（active）'])
                times = list(group['出発時刻'])
                # print('type time', type(times), times)
                # print('type act', type(acts), acts)
                # sys.exit()

                # all_count += 1
                # if acts[-1] != 110:
                #     if acts[-1] == 998 or acts[-1] == 999:
                #         acts[-1] = 110
                #     # print('kitakushitenai!!!')
                # if acts[-1] != 110:
                #     nokitaku_count += 1
                #     continue
                # if acts[0] == 110:
                #     nokitaku_count += 1
                #     continue
                all_count += 1
                # 最後が帰宅（300）もしくは帰社帰校でない場合
                if acts[-1] != 300 and acts[-1] != 201:
                    if acts[-1] == 500 or acts[-1] == 501 or acts[-1] == 999:
                        acts[-1] = 300
                        # continue
                    # print('kitakushitenai!!!')
                if acts[-1] != 300 and acts[-1] != 201:
                    nokitaku_count += 1
                    continue
                # 最初から帰宅の人
                if acts[0] == 300 or acts[0] == 201:
                    nokitaku_count += 1
                    continue
                times = list(group['出発時刻'].dt.floor('h'))

                acts.insert(0, '<b>')
                acts.append('<e>')
                times.insert(0, '<b>')
                times.append('<e>')

                if user_id not in list(user_log_dict.keys()):
                    user_log_dict[user_id] = len(group)
                    user_count_dict[user_id] = 1
                    user_time_traj[user_id] = times
                    user_act_traj[user_id] = acts

                else:
                    user_log_dict[user_id] += len(group)
                    user_count_dict[user_id] += 1
                    user_time_traj[user_id] += times
                    user_act_traj[user_id] += acts

print('nokitaku_count', nokitaku_count)
print('all_count', all_count)

print(user_log_dict, len(user_log_dict))
print(user_count_dict, len(user_count_dict))

new_dict = {key: value // 5 for key, value in user_count_dict.items()}
print(new_dict)
print(sum(new_dict.values()))
print('user_log_dict', user_act_traj)
print('user_time_traj', user_time_traj)
len_values = list(user_time_traj.values())
max_len = max(len(value) for value in len_values)
print('max_len', max_len)
df_time_traj_df = pd.DataFrame.from_dict(user_time_traj, orient='index')
# print('df_time_traj_df', df_time_traj_df)
df_act_traj_df = pd.DataFrame.from_dict(user_act_traj, orient='index')
df_act_traj_df.insert(0, 'user_id', df_act_traj_df.index)
df_time_traj_df.insert(0, 'user_id', df_time_traj_df.index)
# print('df_act_traj_df', df_act_traj_df)

# df_act_traj_df.to_csv(os.path.join(base_path, 'user_act_traj_df.csv'), index=False)
df_act_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_toyosu.csv', index=False)
df_time_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_toyosu.csv', index=False)

# for key, value in user_time_traj.items():
#     print(len(value))

    # print('user_id', key)
    # print('user_time_traj', value)
    # print('user_act_traj', user_act_traj[key])
    # sys.exit()

['.DS_Store', '10004_20190707', '10004_20190708', '10004_20190709', '10004_20190710', '10004_20190711', '10004_20190712', '10004_20190713', '10004_20190714', '10004_20190715', '10004_20190716', '10004_20190717', '10004_20190718', '10004_20190719', '10004_20190720', '10004_20190721', '10004_20190722', '10004_20190723', '10004_20190724', '10004_20190725', '10004_20190726', '10004_20190727', '10004_20190728', '10004_20190729', '10004_20190730', '10004_20190731', '10004_20190801', '10004_20190802', '10004_20190803', '10004_20190804', '10004_20190805', '10004_20190806', '10004_20190807', '10004_20190808', '10004_20190809', '10004_20190810', '10004_20190811', '10004_20190812', '10004_20190813', '10004_20190814', '10004_20190815', '10004_20190816', '10004_20190817', '10004_20190818', '10004_20190819', '10004_20190820', '10004_20190821', '10004_20190822', '10004_20190823', '10004_20190824', '10004_20190825', '10004_20190826', '10004_20190827', '10004_20190828', '10004_20190829', '10004_2019083

渋谷2021PPも利用

In [49]:
import sys
import pandas as pd
import numpy as np
import os
base_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/99_202110/logs'
folderlist = os.listdir(base_path)
user_log_dict = {}
user_count_dict = {} # 何日登場したかのカウント
user_time_traj = {} # key: userid, value: time trajectory
user_act_traj = {} # key: userid, value: act trajectory
nokitaku_count = 0
all_count = 0
folderlist = sorted(folderlist)
print(folderlist)

for folder in folderlist:
    if not os.path.isdir(os.path.join(base_path, folder)):
        continue
    # print(folder)
    filelist = os.listdir(os.path.join(base_path, folder))
    for file in filelist:
        if file.endswith('trip.csv'):
            # print(folder, file)
            df = pd.read_csv(os.path.join(base_path, folder, file), encoding='shift-jis')
            df['出発時刻'] = pd.to_datetime(df['出発時刻']) #, format='%Y/%m/%d %H:%M:%S')
            df['到着時刻'] = pd.to_datetime(df['到着時刻']) #, format='%Y/%m/%d %H:%M:%S')

            grouped = df.groupby('ユーザーID')
            for user_id, group in grouped:
                userid = group['ユーザーID'].values[0]
                group = group.sort_values(by='出発時刻')
                acts = list(group['目的コード（active）'])
                times = list(group['出発時刻'])
                # print('type time', type(times), times)
                # print('type act', type(acts), acts)
                # sys.exit()

                all_count += 1
                # 最後が帰宅（300）もしくは帰社帰校でない場合
                if acts[-1] != 300 and acts[-1] != 201:
                    if acts[-1] == 500 or acts[-1] == 501 or acts[-1] == 999:
                        acts[-1] = 300
                        # continue
                    # print('kitakushitenai!!!')
                if acts[-1] != 300 and acts[-1] != 201:
                    nokitaku_count += 1
                    continue
                # 最初から帰宅の人
                if acts[0] == 300 or acts[0] == 201:
                    nokitaku_count += 1
                    continue
                times = list(group['出発時刻'].dt.floor('h'))

                acts.insert(0, '<b>')
                acts.append('<e>')
                times.insert(0, '<b>')
                times.append('<e>')

                if user_id not in list(user_log_dict.keys()):
                    user_log_dict[user_id] = len(group)
                    user_count_dict[user_id] = 1
                    user_time_traj[user_id] = times
                    user_act_traj[user_id] = acts

                else:
                    user_log_dict[user_id] += len(group)
                    user_count_dict[user_id] += 1
                    user_time_traj[user_id] += times
                    user_act_traj[user_id] += acts

print('nokitaku_count', nokitaku_count)
print('all_count', all_count)

print(user_log_dict, len(user_log_dict))
print(user_count_dict, len(user_count_dict))

new_dict = {key: value // 5 for key, value in user_count_dict.items()}
print(new_dict)
print(sum(new_dict.values()))
print('user_log_dict', user_act_traj)
print('user_time_traj', user_time_traj)
len_values = list(user_time_traj.values())
max_len = max(len(value) for value in len_values)
print('max_len', max_len)
df_time_traj_df = pd.DataFrame.from_dict(user_time_traj, orient='index')
# print('df_time_traj_df', df_time_traj_df)
df_act_traj_df = pd.DataFrame.from_dict(user_act_traj, orient='index')

df_act_traj_df.insert(0, 'user_id', df_act_traj_df.index)
df_time_traj_df.insert(0, 'user_id', df_time_traj_df.index)
# print('df_act_traj_df', df_act_traj_df)

# df_act_traj_df.to_csv(os.path.join(base_path, 'user_act_traj_df.csv'), index=False)
df_act_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_shibu21.csv', index=False)
df_time_traj_df.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_shibu21.csv', index=False)

# for key, value in user_time_traj.items():
#     print(len(value))

    # print('user_id', key)
    # print('user_time_traj', value)
    # print('user_act_traj', user_act_traj[key])
    # sys.exit()

['.DS_Store', '10024_20211001', '10024_20211002', '10024_20211003', '10024_20211004', '10024_20211005', '10024_20211006', '10024_20211007', '10024_20211008', '10024_20211009', '10024_20211010', '10024_20211011', '10024_20211012', '10024_20211013', '10024_20211014', '10024_20211015', '10024_20211016', '10024_20211017', '10024_20211018', '10024_20211019', '10024_20211020', '10024_20211021', '10024_20211022', '10024_20211023', '10024_20211024', '10024_20211025', '10024_20211026', '10024_20211027', '10024_20211028', '10024_20211029', '10024_20211030', '10024_20211031', '10024_20211101', '10024_20211102', '10024_20211103', '10024_20211104', '10024_20211105', '10024_20211106', '10024_20211107', '10024_20211108', '10024_20211109', '10024_20211110', '10024_20211111', '10024_20211112', '10024_20211113', '10024_20211114', '10024_20211115', '10024_20211116', '10024_20211117', '10024_20211118', '10024_20211119', '10024_20211120', '10024_20211121', '10024_20211122', '10024_20211123', '10024_2021112

生成された各データ列から1週間分でデータを分割する

In [50]:
df_shibu22old_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog01.csv')
df_shibu22old_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog01.csv')
df_shibu22new_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog02.csv')
df_shibu22new_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog02.csv')
df_shibu23_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog03.csv')
df_shibu23_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog03.csv')
df_toyosu_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_toyosu.csv')
df_toyosu_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_toyosu.csv')
df_shibu21_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_shibu21.csv')
df_shibu21_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_shibu21.csv')

ここからデータを絞る．まずはカテゴリ値を設定

In [51]:
import os

act_dict_shibuya2223 = {
    110: 1, # H
    200: 2, # S
    210: 2, # S
    220: 1, # hotel->home
    300: 3, # W 
    310: 4, # W2
    400: 3, # W
    998: 5, # Other
    999: 5 # , # Other
    # '<b>': '<b>',
    # '<e>': '<e>'

}

act_dict_shibuya21toyosu = {
    300: 1, # H
    201: 1, # H
    400: 2, # S
    401: 2, 
    402: 2, 
    404: 2, 
    405: 2, 
    406: 2, 
    100: 3, # W
    200: 4, # W2
    403: 5, # Ohter
    407: 5, # Ohter
    500: 5, # Ohter
    501: 5, # Ohter
    999: 5 #, # Ohter`
    # '<b>': '<b>',
    # '<e>': '<e>'
}


type_list = ['01', '02', '03', '_toyosu', '_shibu21']
for type_ in type_list:
    df_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '.csv')
    # df_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog' + type_ + '.csv')
    # df_act_2 = df_act.copy()
    # df_act_2 = df_act_2.iloc[:, :5]
    # 変換辞書の分別
    if type_ == '_toyosu' or type_ == '_shibu21':
        act_dict = act_dict_shibuya21toyosu
    elif type_ == '01' or type_ == '02' or type_ == '03':
        act_dict = act_dict_shibuya2223
    
    # まず全部置換する
    # df_act = df_act.applymap(lambda x: act_dict.get(x, x) if isinstance(x, int) else x)

    def try_convert_int(x):
        if x in ['<b>', '<e>']:
            return x
        try:
            return int(float(x))  # 300.0 → 300 などにも対応
        except:
            return x  # 変換できなければそのまま

    df_act = df_act.applymap(try_convert_int)
    df_act = df_act.applymap(lambda x: act_dict.get(x, x))

    # print('df4type = ', df_act_2.iloc[:, 4], 'val:', df_act_2.iloc[3, 4], 'type', type(df_act_2.iloc[2, 4]))
    # df_act_2 = df_act_2.applymap(lambda x: act_dict.get(x, x) if isinstance(x, int) else x)
    df_act.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '_new.csv', index=False)
    # df_act_2.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '_new_2.csv', index=False)
    print('finished')
    # sys.exit()

/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act = df_act.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act = df_act.applymap(lambda x: act_dict.get(x, x))
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act = df_act.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act = df_act.applymap(lambda x: act_dict.get(x, x))
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use D

finished
finished
finished
finished
finished


/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act = df_act.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/3635437672.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act = df_act.applymap(lambda x: act_dict.get(x, x))


続いて同一活動種類の連続を排除

In [55]:
import pandas as pd

def remove_consecutive_duplicates(act_row, time_row):
    new_act = [act_row[0]]
    new_time = [time_row[0]]
    for i in range(1, len(act_row)):
        if act_row[i] != act_row[i-1]:
            new_act.append(act_row[i])
            new_time.append(time_row[i])
    return pd.Series(new_act), pd.Series(new_time)

# 読み込み
type_list = ['01', '02', '03', '_toyosu', '_shibu21']
for type_ in type_list:
    df_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '_new.csv', header=None) # ここまではuseridが入ってる
    df_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog' + type_ + '.csv', header=None)

    # 最大列数（不足分を埋める用）
    max_len = max(df_act.shape[1], df_time.shape[1])

    # NaNを埋める（列数を揃える）
    df_act = df_act.reindex(columns=range(max_len))
    df_time = df_time.reindex(columns=range(max_len))

    # 各行について処理
    act_cleaned = []
    time_cleaned = []

    for i in range(1, len(df_act)):
        a_row = df_act.iloc[i].tolist()
        t_row = df_time.iloc[i].tolist()
        a_new, t_new = remove_consecutive_duplicates(a_row, t_row)
        act_cleaned.append(a_new)
        time_cleaned.append(t_new)

    # データフレーム化して保存
    df_act_cleaned = pd.DataFrame(act_cleaned)
    df_time_cleaned = pd.DataFrame(time_cleaned)

    df_act_cleaned.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '_edited.csv', index=False, header=False)
    df_time_cleaned.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog' + type_ + '_edited.csv', index=False, header=False)


この次に1週間ごとのデータに分割（pp_data_seikei.py）

5つのデータをマージする

の前に長さ揃ってなかったので揃える．．

In [62]:
import csv

type_list = ['01', '02', '03', '_toyosu', '_shibu21']
for type_ in type_list:
    path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '_for_input.csv'
    with open(path, newline='') as f:
        # lines = f.readlines()
        reader = csv.reader(f)
        max_cols = max(len(row) for row in reader)
        print('max_cols', max_cols)
        

max_cols 246
max_cols 91
max_cols 184
max_cols 691
max_cols 527


In [63]:
for type_ in type_list:
    path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog' + type_ + '_for_input.csv'
    with open(path, newline='') as f:
        reader = csv.reader(f)
        rows = list(reader)
    # 2. 各行の空要素・None・空白文字列を除外
    cleaned_rows = [[cell for cell in row if cell not in [None, '', 'NaN']] for row in rows]

    # 3. 必要であれば DataFrame にはせずリストで扱う（非矩形）
    # 例えば：確認用
    for row in cleaned_rows[:5]:
        print(row)

    # 4. 保存するなら：1行ずつCSV形式で出力（列数バラバラOK）
    output_path = path.replace('.csv', '_cleaned.csv')
    with open(output_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(cleaned_rows)

    with open(output_path, newline='') as f:
        reader = csv.reader(f)
        rows = list(reader)
        # 最も長い行の長さを取得
        max_cols = max(len(row) for row in rows)
        print('max_cols', max_cols)

    

['23002', '<b>', '3', '5', '2', '3', '2', '1', '<e>', '<b>', '3', '2', '3', '2', '1', '<e>', '<b>', '3', '2', '3', '2', '5', '2', '1', '<e>', '<b>', '3', '2', '3', '2', '1', '<e>', '<b>', '3', '2', '3', '2', '1', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<e>']
['23002', '<b>', '2', '1', '3', '2', '1', '<e>', '<b>', '3', '2', '3', '2', '5', '2', '1', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<e>']
['23003', '<b>', '3', '1', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '3', '1', '<e>', '<b>', '955', '<e>', '<b>', '2', '1', '<e>', '<b>', '2', '1', '<e>', '<e>']
['23003', '<b>', '2', '1', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<e>']
['23003', '<b>', '2', '1', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '955', '<e>', '<b>', '2', '1', '<e>', '<e>']
max_cols

In [64]:
import csv

type_list = ['01', '02', '03', '_toyosu', '_shibu21']
act_all_rows = [] # 付け足す
time_all_rows = [] # 付け足す

act_max_cols = 0
time_max_cols = 0

for type_ in type_list:
    act_path = f'/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog{type_}_for_input.csv'
    time_path = f'/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog{type_}_for_input.csv'

    # 読み込み + 空文字・None・"NaN"除去
    with open(act_path, newline='') as f:
        reader = csv.reader(f)
        act_cleaned_rows = [[cell for cell in row if cell not in [None, '', 'NaN']] for row in reader]
    
    with open(time_path, newline='') as f:
        reader = csv.reader(f)
        time_cleaned_rows = [[cell for cell in row if cell not in [None, '', 'NaN']] for row in reader]

    # 最大列数を更新
    for row in act_cleaned_rows:
        act_max_cols = max(act_max_cols, len(row))
    
    for row in time_cleaned_rows:
        time_max_cols = max(time_max_cols, len(row))

    act_all_rows.extend(act_cleaned_rows)
    time_all_rows.extend(time_cleaned_rows)

# 行の長さを最大列数にそろえ、<p>で埋める
act_padded_rows = [row + ['<p>'] * (act_max_cols - len(row)) for row in act_all_rows]
time_padded_rows = [row + ['<p>'] * (time_max_cols - len(row)) for row in time_all_rows]

# 結合結果を保存
act_output_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_finalinput.csv' # これが最終的なインプットになる
time_output_path = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_finalinput.csv' # これが最終的なインプットになる

with open(act_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(act_padded_rows)

with open(time_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(time_padded_rows)

print(f"✔️ 結合済みファイルを保存: {act_output_path}, {time_output_path}")
print(f"🧱 最大列数: {max_cols}, 総行数: {len(act_padded_rows)}, {len(time_padded_rows)}")

✔️ 結合済みファイルを保存: /Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog_finalinput.csv, /Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_finalinput.csv
🧱 最大列数: 59, 総行数: 3992, 3992


時間の表現に直す

In [68]:
def convert_datetime_to_hour(token):
    try:
        # 時刻が含まれる場合（フォーマット付き日付）
        dt = pd.to_datetime(token, errors='raise')
        if dt.hour == 0 and "00:00:00" not in token:
            return 25 # token  # たとえば "2022-12-21" のような場合
        return str(dt.hour)
    except Exception:
        # 変換できない場合はそのまま返す
        return token
## 
df_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_finalinput.csv', header=None)
df_time = df_time.applymap(lambda x: convert_datetime_to_hour(x) if pd.notna(x) else x)
# converted_row = [convert_datetime_to_hour(tok) for tok in row]


/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_9930/168936303.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_time = df_time.applymap(lambda x: convert_datetime_to_hour(x) if pd.notna(x) else x)


In [69]:
df_time.to_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog_finalinput_hour.csv', index=False, header=False)

In [70]:
print(df_time)

         0    1   2   3    4    5    6    7    8    9   ...   74   75   76  \
0     23002  <b>   8  11   12   12   19   21  <e>  <b>  ...  <p>  <p>  <p>   
1     23002  <b>  10  10   12   18   21  <e>  <b>    8  ...  <p>  <p>  <p>   
2     23003  <b>   6  19  <e>  <b>   25  <e>  <b>   25  ...  <p>  <p>  <p>   
3     23003  <b>  14  16  <e>  <b>   25  <e>  <b>   25  ...  <p>  <p>  <p>   
4     23003  <b>   7  14  <e>  <b>   25  <e>  <b>   25  ...  <p>  <p>  <p>   
...     ...  ...  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3987  21402  <b>   7  13   21  <e>  <b>   25  <e>  <b>  ...  <p>  <p>  <p>   
3988  21402  <b>  18  22  <e>  <b>   25  <e>  <b>   25  ...  <p>  <p>  <p>   
3989  21415  <b>  10  16  <e>  <b>   25  <e>  <b>   25  ...  <p>  <p>  <p>   
3990  21457  <b>   8  15   17  <e>  <b>   25  <e>  <b>  ...  <p>  <p>  <p>   
3991  21550  <b>   9  10   10   10  <e>  <b>   25  <e>  ...  <p>  <p>  <p>   

       77   78   79   80   81   82   83  
0     <p>  <p>  <p>  

In [37]:
# # concatで良い．
# type_list = ['02', '03', '_toyosu', '_shibu21']
# df_act = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog01_for_input.csv')
# print(df_act)
# df_time = pd.read_csv('/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog01_for_input.csv')

# # 一番長い列数は
# print(df_act.shape) 
# sys.exit()

# for type_ in type_list:
#     df_act_add = pd.read_csv(f'/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlog{type_}_for_input.csv')
#     df_time_add = pd.read_csv(f'/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelog{type_}_for_input.csv')
#     df_act = pd.concat([df_act, df_act_add], axis=0)
#     df_time = pd.concat([df_time, df_time_add], axis=0)

# df_act.to_csv(f'/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/actlogall_for_input.csv', index=False, header=False)
# df_time.to_csv(f'/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog/timelogall_for_input.csv', index=False, header=False)
    

ParserError: Error tokenizing data. C error: Expected 30 fields in line 4, saw 165
